In [1]:
################ Letter-Recognition #################
# Disable info messages from the tesnorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
################## Import the libraries #####################
import numpy as np
import pandas as pd
import keras
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D,MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,scale
from sklearn.metrics import accuracy_score
from scipy.ndimage import zoom

In [2]:
############### Load the dataset ###################
path = 'HIGGS.csv'
classes = 2
data=pd.read_csv(path, header=None)
samples = data.sample(n=700000)
print(samples)

          0         1         2         3         4         5         6   \
6974     0.0  0.316789 -0.988632  0.431544  2.229713 -0.760718  1.019955   
1294416  1.0  0.463196  0.710943 -0.217192  1.744388  1.272113  0.810083   
6031723  0.0  0.343874  0.510305  1.591204  0.632275  0.324807  0.730659   
6585226  1.0  0.419457 -0.071154 -0.789811  1.177361  0.791589  0.662778   
1395076  0.0  0.377731  0.728474 -0.782598  0.740253  0.779623  0.765012   
...      ...       ...       ...       ...       ...       ...       ...   
5480741  0.0  0.852273  1.631344  0.428215  2.291045  0.642070  0.802388   
9413401  0.0  0.768272  0.777173  1.670549  0.983377  0.162802  0.690810   
8188728  1.0  0.678415  0.601858  0.174643  1.001682 -0.228374  1.348733   
3354899  1.0  0.564034 -0.769490 -1.268657  0.287806 -0.081558  0.539841   
5525977  1.0  0.893999  1.075207  1.261062  0.923675  0.605190  0.868986   

               7         8         9   ...        19        20        21  \
6974     0.

In [4]:
X =samples.iloc[:,1:]
Y =samples.iloc[:,0]
X = MinMaxScaler().fit_transform(X)
classes = 2
Y = to_categorical(Y,classes)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(560000, 28) (560000, 2)
(140000, 28) (140000, 2)


In [6]:
n = X_train.shape[1]
dim = X_train.shape[1]
final_dim = min(32*32,n*n)
# convert 2-D to 3-D.
while dim<final_dim:
  features = int(dim/2)
  #f = feature-60
  model = Sequential()
  if (dim+features)>final_dim:
    features = final_dim - dim
    model.add(Dense(features,name='feature', activation='relu',input_shape=(dim,)))
  else:
    model.add(Dense(features,name='feature', activation='relu',input_shape=(dim,)))
  model.add(Dense(classes,activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam')
  model.fit(X_train,Y_train,epochs=1,batch_size=512,verbose=1)
  extract = Model(model.inputs, model.get_layer('feature').output)
  features1 = extract.predict(X_train)
  features2 = extract.predict(X_test)
  X_train = np.concatenate((X_train, features1), axis=1) # concatenate on horizontal axis
  X_test = np.concatenate((X_test, features2), axis=1)
  dim = X_train.shape[1]
print(X_train.shape,X_test.shape)
s = MinMaxScaler()
X_train = s.fit_transform(X_train)
X_test = s.transform(X_test)

1094/1094 [==============================] - 2s 2ms/step - loss: 0.6171
(560000, 784) (140000, 784)


In [7]:
from PIL import Image
x_train = []
x_test = []
for i in range(len(X_train)):
    img = Image.fromarray(X_train[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_train.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_train = np.array(x_train,dtype='float32')
for i in range(len(X_test)):
    img = Image.fromarray(X_test[i])
    imgs = img.resize(size=(32, 32))
    imgs = np.array(imgs)
    x_test.append(np.repeat(imgs[:, :, np.newaxis], 3, axis=2))
x_test = np.array(x_test,dtype='float32')
print(x_train.shape,x_test.shape)

(560000, 32, 32, 3) (140000, 32, 32, 3)


In [8]:
############### Feature-extraction ###############
# pretrained model for FE
pretrained_model = keras.applications.DenseNet121(input_shape=(32,32,3), weights=None, include_top=False)
x = Flatten()(pretrained_model.output)
x = Dense(512, activation='relu',name='flattenn')(x)
x = Dropout(0.5)(x)
prediction = Dense(classes, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=prediction)
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# fit training data and validate on the testing data
model.fit(x_train,Y_train,batch_size=256, epochs=10,verbose=1,validation_data=(x_test,Y_test))

Epoch 1/10
2188/2188 [==============================] - 158s 72ms/step - loss: 0.6256 - accuracy: 0.6540 - val_loss: 0.7184 - val_accuracy: 0.4828
Epoch 2/10
2188/2188 [==============================] - 153s 70ms/step - loss: 0.6210 - accuracy: 0.6548 - val_loss: 0.6202 - val_accuracy: 0.6522
Epoch 3/10
2188/2188 [==============================] - 157s 72ms/step - loss: 0.6192 - accuracy: 0.6563 - val_loss: 0.6525 - val_accuracy: 0.6088
Epoch 4/10
2188/2188 [==============================] - 157s 72ms/step - loss: 0.6191 - accuracy: 0.6564 - val_loss: 0.6181 - val_accuracy: 0.6540
Epoch 5/10
2188/2188 [==============================] - 157s 72ms/step - loss: 0.6178 - accuracy: 0.6565 - val_loss: 0.6188 - val_accuracy: 0.6532
Epoch 6/10
2188/2188 [==============================] - 157s 72ms/step - loss: 0.6174 - accuracy: 0.6565 - val_loss: 0.6181 - val_accuracy: 0.6548
Epoch 7/10
2188/2188 [==============================] - 158s 72ms/step - loss: 0.6170 - accuracy: 0.6577 - val_loss: 0

In [9]:
score = model.evaluate(x_test,Y_test)
acc = model.evaluate(x_train,Y_train)
print('training accuracy: ',acc[1])
print('testing accuracy: ',score[1])

17500/17500 [==============================] - 166s 9ms/step - loss: 0.6164 - accuracy: 0.6540
training accuracy:  0.6540499925613403
testing accuracy:  0.6532214283943176
